In [17]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Input, Multiply, Add, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

def apply_attention(input_tensor):
    avg_pool = GlobalAveragePooling2D()(input_tensor)
    dense1 = Dense(64, activation='relu')(avg_pool)
    dense2 = Dense(input_tensor.shape[-1], activation='sigmoid')(dense1)
    attention_output = Multiply()([input_tensor, dense2])
    return attention_output

def residual_block(input_tensor, filters, kernel_size=(3, 3), stride=(1, 1)):
    residual = input_tensor
    x = Conv2D(filters, kernel_size, strides=stride, padding='same', activation='relu')(input_tensor)
    x = Conv2D(filters, kernel_size, strides=stride, padding='same', activation='relu')(x)
    x = Add()([x, residual])
    return x

def custom_cnn_model(input_shape=(224, 224, 3), num_classes=10):
    inputs = Input(shape=input_shape)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = residual_block(x, 64)
    x = apply_attention(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs, outputs)
    return model

def load_data_and_labels(data_path, target_size=(224, 224)):
    images = []
    labels = []
    category_names = []

    for domain_folder in os.listdir(data_path):
        domain_path = os.path.join(data_path, domain_folder)
        if os.path.isdir(domain_path):  
            for category_folder in os.listdir(domain_path):
                category_path = os.path.join(domain_path, category_folder)
                if os.path.isdir(category_path):  
                    for image_file in os.listdir(category_path):
                        image_path = os.path.join(category_path, image_file)
                        image = cv2.imread(image_path)
                        if image is not None:
                            resized_image = cv2.resize(image, target_size)
                            images.append(resized_image)
                            labels.append(category_folder)
                            if category_folder not in category_names:
                                category_names.append(category_folder)

    images = np.array(images)
    labels = np.array(labels)
    category_to_index = {name: idx for idx, name in enumerate(category_names)}
    labels = np.array([category_to_index[label] for label in labels])

    return images, labels, category_names

def split_data_by_category(images, labels, test_ratio=0.5):
    train_images, train_labels, test_images, test_labels = [], [], [], []

    for label in np.unique(labels):
        indices = np.where(labels == label)[0]
        category_images, category_labels = images[indices], labels[indices]
        images_train, images_test, labels_train, labels_test = train_test_split(
            category_images, category_labels, test_size=test_ratio, random_state=42
        )
        train_images.extend(images_train)
        train_labels.extend(labels_train)
        test_images.extend(images_test)
        test_labels.extend(labels_test)

    return np.array(train_images), np.array(train_labels), np.array(test_images), np.array(test_labels)



In [18]:
data_path = "/Users/yanzhu/Documents/Office31"
images, labels, category_names = load_data_and_labels(data_path)
train_images, train_labels, test_images, test_labels = split_data_by_category(images, labels, test_ratio=0.5)
input_shape = (224, 224, 3)
num_classes = len(category_names)
custom_cnn_model =custom_cnn_model(input_shape=input_shape, num_classes=num_classes)

custom_cnn_model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

custom_cnn_model.fit(train_images, train_labels, epochs=10, batch_size=32, validation_data=(test_images, test_labels))

Epoch 1/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 34s 577ms/step - accuracy: 0.0396 - loss: 209.5890 - val_accuracy: 0.0353 - val_loss: 3.4316
Epoch 2/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 33s 574ms/step - accuracy: 0.0358 - loss: 3.4302 - val_accuracy: 0.0353 - val_loss: 3.4223
Epoch 3/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 32s 570ms/step - accuracy: 0.0521 - loss: 3.4047 - val_accuracy: 0.0733 - val_loss: 3.3612
Epoch 4/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 32s 567ms/step - accuracy: 0.0757 - loss: 3.3573 - val_accuracy: 0.0943 - val_loss: 3.3560
Epoch 5/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 33s 575ms/step - accuracy: 0.1431 - loss: 3.2008 - val_accuracy: 0.1549 - val_loss: 3.3679
Epoch 6/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 32s 567ms/step - accuracy: 0.3034 - loss: 2.5958 - val_accuracy: 0.1963 - val_loss: 3.3143
Epoch 7/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 32s 568ms/step - accuracy: 0.5323 - loss: 1.7911 - val_accuracy: 0.2216 - val_loss: 3.9547
Epoch 8/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 32s 569ms/step - accuracy: 0.6476 - loss: 1.3253 - val_ac